In [119]:
import os
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score

# get version
print("pandas version",pd.__version__)

fp = "G:\\My Drive\\Darby Work\\XRF fundamentals vs. MVA\\"

pandas version 1.3.2


In [92]:
spectra_path = "Z:\\data_pXRF\\MHC_Olympus_spectra.csv"
spectra = pd.read_csv(spectra_path)

metadata_path = "Z:\\data_pXRF\\MHC_Olympus_metadata.csv"
metadata = pd.read_csv(metadata_path)
comps = metadata.drop(metadata.columns[1:8], axis=1)
comps.columns = comps.columns.map(lambda x: x.split()[0])

In [3]:
# select samples with olympus predictions
pred_samples = list(metadata[
    metadata['Olympus-Predicted'] == 'yes'
]['pkey'])

# add wave column
pred_samples.insert(0, 'wave')


pred_spectra = spectra.filter(
    items = pred_samples
)

In [4]:
o1_spectra = pred_spectra.filter(regex=('(_1$)|(wave)'), axis=1)
o2_spectra = pred_spectra.filter(regex=('(_2$)|(wave)'), axis=1)

## Sensitivity

In [5]:
noise_path = fp+"Braga noise regions.csv"
noise = pd.read_csv(noise_path)

In [6]:
print("Datasets cleaned:")
count = 0
for df in tqdm([o1_spectra, o2_spectra]):
   
    if count == 0:
        noise_temp = noise[noise['Filter'] == 1].reset_index(drop=True)
    else: noise_temp = noise[noise['Filter'] == 2].reset_index(drop=True)
    count += 1
        
    print("Rows cleaned:") 
    for row in tqdm(df.index):
        nm = df['wave'][row]
        # remove rows below first region
        if nm < noise_temp['Start'][0]:
            df.drop(row, axis = 'index', inplace=True)
        # remove rows after last region
        elif nm > noise_temp['Stop'][len(noise_temp)-1]:
            df.drop(row, axis = 'index', inplace=True)
        # remove rows between the regions
        for region in range(len(noise_temp)-1):
            if (nm > noise_temp['Stop'][region]) & (nm < noise_temp['Start'][region+1]):
                df.drop(row, axis = 'index', inplace=True)

Datasets cleaned:


  0%|          | 0/2 [00:00<?, ?it/s]

Rows cleaned:


  0%|          | 0/2042 [00:00<?, ?it/s]

C:\Users\ytsma22c\Anaconda3\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Rows cleaned:


  0%|          | 0/2042 [00:00<?, ?it/s]

In [7]:
o1_sens = round(o1_spectra.set_index('wave').std(axis=1).mean(),3)
o2_sens = round(o2_spectra.set_index('wave').std(axis=1).mean(),3)

In [8]:
print("Filter 1 sensitivity:", o1_sens, 
      "\nFilter 2 sensitivity:",o2_sens)

Filter 1 sensitivity: 261.024 
Filter 2 sensitivity: 159.383


### Function to calculate LBDQs from model and instrument sensitivity

In [112]:
def get_lbdq(folder, file_list):
     
    print("LBDQ:")
    
    coeffs = []
    elem_list = []
    filt_list = []
    lob_list = []
    lod_list = []
    loq_list = []
    
    for filter_n in ['O1', 'O2']:
        
        # get sensitivity value
        sensitivity = o1_sens if filter_n == 'O1' else o2_sens
    
        ftype = filter_n + "_coeff"

        # read models
        for file in tqdm(file_list):
            if ftype in file:       
                path = folder + file
                data = pd.read_csv(path, skiprows = [0])

                # convert to dataframe
                data = data.T

                # adapt to different element naming b/w datasets
                data.columns = data.iloc[0].map(lambda x: x.split()[0])
                data = data.drop(data.index[0])
                element = data.columns[0]

                # populate lists
                elem_list.append(element)
                filt_list.append(filter_n)

                # calculate regression vectors
                vector = pow(data, 2).sum().pow(.5)  #square root of sum of squares
                
                # calculate values
                factors = {
                    'LOB' : 1.645,
                    'LOD' : 3.3,
                    'LOQ' : 10
                }

                lob_list.append(factors['LOB'] * sensitivity * vector[0])
                lod_list.append(factors['LOD'] * sensitivity * vector[0])
                loq_list.append(factors['LOQ'] * sensitivity * vector[0])

    # make dataframe
    df = pd.DataFrame({
        'element' : elem_list,
        'filter' : filt_list,
        'LOB' : lob_list,
        'LOD' : lod_list,
        'LOQ' : loq_list
    })
    
    # change col formats
    cols = df.columns.drop(['element', 'filter'])
    df[cols] = df[cols].apply(pd.to_numeric)
    
    return df

### Function to calculate RMSEPs

In [121]:
majors = ['Al2O3', 'CaO', 'Fe2O3', 'MgO', 'MnO', 'P2O5', 'SiO2', 'TiO2']

def get_rmsep(folder, file_list, lbdq):
    
    print("RMSEP:")
    
    elem_list = []
    filt_list = []
    avg_list = []
    rmsep_list = []
    r2_list = []
    
    for filter_n in ['O1', 'O2']:
    
        ftype = filter_n + "_test"

        for file in tqdm(file_list):
            if ftype in file:       
                path = (folder + file)
                data = pd.read_csv(path)

                # get element
                element = data.columns[1].split()[0]
                elem_list.append(element)
                filt_list.append(filter_n)

                # format columns
                data.columns = ['pkey', 'Actual', 'Pred']
                data = data.drop([0])
                data.Pred = data.Pred.astype(float)  

                # remove predictions above 100 for majors
                if element in majors:
                    data = data[data.Pred < 100]

                # remove all predictions below 0
                data = data[data.Pred > 0].reset_index(drop=True).sort_index(axis=1)

                # order columns
                data = data[['pkey', 'Actual', 'Pred']].drop_duplicates(subset = 'pkey').sort_values(by='pkey').reset_index(drop=True)

                # subselect relevant reference values
                ref = lbdq[(lbdq['element'].astype(str) == element) &
                           (lbdq['filter'].astype(str) == filter_n)].reset_index(drop=True)

                # add in Actual concentrations
                temp_comps = comps[comps.pkey.isin(data.pkey)].reset_index(drop=True)                   
                data['Actual'] = temp_comps[temp_comps['pkey'] == data['pkey']][element]

                # remove NaN Actual values....which idk why they'd be there
                data = data.dropna()

                # calculate values
                loq = ref['LOQ'].iloc[0]
                # select just predictions above the LOQ
                data = data[data.Pred > loq].reset_index(drop=True)
                # get average concentration
                avg = data['Actual'].mean()
                avg_list.append(avg)
                # get R2
                r2 = r2_score(data.Actual, data.Pred)
                r2_list.append(r2)
                # get RMSE-P
                data['sqerror'] = (data.Actual - data.Pred).pow(2)
                rmsep = data['sqerror'].mean() ** 0.5
                rmsep_list.append(rmsep)

    
    df = pd.DataFrame({
        "element" : elem_list,
        "filter" : filt_list,
        "Avg" : avg_list,
        "RMSEP" : rmsep_list,
        "R2" : r2_list,
    })
    
    
    return df

In [122]:
def get_results(regression, n_range):
    
    print('Calculating for', regression, n_range)

    folder = fp+"\\models\\"+regression+"\\"+n_range+"\\"
    file_list = os.listdir(folder)

    # calculate lbdq
    lbdq = get_lbdq(folder, file_list)
    # calculate rmsep with lbdq results
    rmsep = get_rmsep(folder, file_list, lbdq)
    # merge results
    df = pd.merge(lbdq, rmsep, how='outer', on=['element', 'filter'])
    df.insert(loc=2, column='num_range', value=n_range)
    df.insert(loc=2, column='regression', value=regression)
    # return full results
    return df 

In [123]:
get_results('lasso', '0-750')

Calculating for lasso 0-750
LBDQ:


  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

RMSEP:


  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

,element,filter,regression,num_range,LOB,LOD,LOQ,Avg,RMSEP,R2
0,Al2O3,O1,lasso,0-750,1.920544,3.852762,11.675038,14.284519,3.304214,-0.211958
1,CaO,O1,lasso,0-750,2.966556,5.951145,18.033773,31.630000,13.528613,0.249996
2,Fe2O3,O1,lasso,0-750,2.308474,4.630982,14.033278,NaN,NaN,NaN
3,Al2O3,O2,lasso,0-750,0.619296,1.242356,3.764715,13.907448,1.424941,0.815621
4,CaO,O2,lasso,0-750,0.156545,0.314041,0.951639,7.679554,2.103881,0.868282
